<a href="https://colab.research.google.com/github/sabitech1/generative_ai/blob/main/Function_Calling_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Required pip installs

In [1]:
!pip install openai finnhub-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00


In [2]:
!pip install google-cloud-secret-manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 3.0 MB/s eta 0:00:00


#  Example - GPT Function calling for Stock Prices


In [3]:
import openai
import os
import json
import finnhub
from google.cloud import secretmanager

## Retrieve API keys from GCP for OpenAI & Finnhub
First retrieve credentials as secret from GCP

In [9]:
from google.colab import auth
auth.authenticate_user()

In [10]:
def get_secret_from_gcp(secret_name):
    client=secretmanager.SecretManagerServiceClient()
    project_id="<your-project-id-here>"
    resource_name=f"projects/{project_id}/secrets/{secret_name}/versions/latest"
    # Get OpenAI key as secret from GCP
    response=client.access_secret_version(name=resource_name)
    secret=response.payload.data.decode('utf-8')
    return secret


In [11]:

openai.api_key=get_secret_from_gcp("OPENAI_API_KEY")
FINNHUB_API_KEY=get_secret_from_gcp("FINNHUB_API_KEY")




In [12]:
stock_prx_response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the current price of GOOG"},
    ]
)

In [13]:
stock_prx_response

<OpenAIObject chat.completion id=chatcmpl-7tT4Z1ZBSRQTbn0KzONFQhxe0UlPd at 0x79bb34324fe0> JSON: {
  "id": "chatcmpl-7tT4Z1ZBSRQTbn0KzONFQhxe0UlPd",
  "object": "chat.completion",
  "created": 1693454151,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I'm sorry, but I don't have real-time data. To find the current price of Google (GOOG) stock, you can check financial news websites, stock market apps, or contact your broker for the most up-to-date information."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 25,
    "completion_tokens": 49,
    "total_tokens": 74
  }
}

## Define function to get stock price and test

In [14]:
def get_latest_stock_price(stock_symbol):
  finnhub_client=finnhub.Client(api_key=FINNHUB_API_KEY)
  px_key='c'
  latest_prx = finnhub_client.quote(stock_symbol)[px_key]
  return latest_prx

In [15]:
print(get_latest_stock_price("AAPL"))

187.65


In [16]:
get_stock_price_func = {
    "name": "get_latest_stock_price",
    "description" : "Get latest stock price for symbol",
    "parameters" : {
      "type":"object",
      "properties": {
         "stock_symbol" : {
   	        "type": "string",
	          "description" : "The stock symbol to look up latest price for"
	        }
	      },
       "required" : ["stock_symbol"]
       }
      }

### Call using auto option

In [17]:
prompt="Get latest stock price for GOOG"
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-0613",
  messages=[
        {"role": "user", "content": prompt},
    ],
  functions=[get_stock_price_func],
  function_call="auto"
)
response

<OpenAIObject chat.completion id=chatcmpl-7tT4r3KN84xVGwO5kHrvhJOBOiVCP at 0x79bb1054ebb0> JSON: {
  "id": "chatcmpl-7tT4r3KN84xVGwO5kHrvhJOBOiVCP",
  "object": "chat.completion",
  "created": 1693454169,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_latest_stock_price",
          "arguments": "{\n  \"stock_symbol\": \"GOOG\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 67,
    "completion_tokens": 19,
    "total_tokens": 86
  }
}

### Call using function name option


In [18]:

prompt="Get latest stock price for GOOG"
fxn_response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-0613",
  messages=[
        {"role": "user", "content": prompt},
    ],
  functions=[get_stock_price_func],
  function_call={"name": "get_latest_stock_price"}
)
fxn_response

<OpenAIObject chat.completion id=chatcmpl-7tT507d0uw1h34OYkohqPiQg89v65 at 0x79bb10528cc0> JSON: {
  "id": "chatcmpl-7tT507d0uw1h34OYkohqPiQg89v65",
  "object": "chat.completion",
  "created": 1693454178,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_latest_stock_price",
          "arguments": "{\n  \"stock_symbol\": \"GOOG\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 75,
    "completion_tokens": 11,
    "total_tokens": 86
  }
}

### Call using none option

In [19]:
prompt="Get latest stock price for GOOG"
response_none = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-0613",
  messages=[
        {"role": "user", "content": prompt},
    ],
  functions=[get_stock_price_func],
  function_call="none"
)
response_none

<OpenAIObject chat.completion id=chatcmpl-7tT56WA4uo9Dj1fDuDJDtcrpTvaK0 at 0x79bb1054f2e0> JSON: {
  "id": "chatcmpl-7tT56WA4uo9Dj1fDuDJDtcrpTvaK0",
  "object": "chat.completion",
  "created": 1693454184,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"stock_symbol\": \"GOOG\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 68,
    "completion_tokens": 11,
    "total_tokens": 79
  }
}

### Call with invalid option 'unknown'



Expected to fail

In [20]:
prompt="Get latest stock price for GOOG"
bad_completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-0613",
  messages=[
        {"role": "user", "content": prompt},
    ],
  functions=[get_stock_price_func],
  function_call="unknown"
)
bad_completion

InvalidRequestError: ignored

We now extract the args and call the function:


## Extract the function name and params from response

In [21]:
output = response.choices[0].message
print(output)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_latest_stock_price",
    "arguments": "{\n  \"stock_symbol\": \"GOOG\"\n}"
  }
}


In [22]:
stock_symbol=json.loads(output.function_call.arguments).get("stock_symbol")
print(stock_symbol)

GOOG


In [23]:
params = json.loads(output.function_call.arguments)

## Calling the function with arguments

We now call the function anonymously :

In [24]:
import json
# extract function
my_func = eval(output.function_call.name)
params = json.loads(output.function_call.arguments)
# call function, unpacking params
stock_price = my_func(**params)
print(f"Stock price = {stock_price}")

Stock price = 136.93


## Now add the function output back to the message and make a 2nd API call

In [25]:
prompt="Get latest stock price for GOOG"
response2 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-0613",
  messages=[
        {"role": "user", "content": prompt},
        {"role": "function", "name": output.function_call.name,"content": str(stock_price)},
    ],
  functions=[get_stock_price_func],
)
content2 = response2.choices[0].message.content
print(content2)

The latest stock price for GOOG (Google) is $136.93.
